In [1]:
from src.model import Model
import torch
from src.config import Config
from src.generate_data import gen_data
from src.train_model import train
from src.evaluate import evaluate

In [2]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"Device set to {device.upper()}")

Device set to MPS


# Define Model

In [3]:
from torch.optim.lr_scheduler import ExponentialLR

number_of_prev_frames = 5
frame_size = 64 * 32

# Model Params
hidden_dim = frame_size
num_layers = 5
input_shape = frame_size * number_of_prev_frames
output_shape = frame_size

model = Model(num_layers, hidden_dim, input_shape, output_shape)

opt = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ExponentialLR(opt, gamma=0.5)
loss = torch.nn.CrossEntropyLoss()

# Define Config File

In [6]:
from pathlib import Path

config = Config(
    run_name = 'one_ball_exp',
    data_storage_dir = Path('/Users/ben/Projects/bouncyBall/data'),
    device=device,
    
    # === Visualization ===
    
    # Number of intervals to show per second. simulation_interval == simulation_fps => Realtime
    simulation_fps = 40,
    # Visualize the data generation progress (slows down data generation significantly).
    visualize_data_gen = True,
    # Visualize the training progress (slows down training significantly).
    visualize_training = False,
    # Skip the visualization of these epochs during training.
    skip_epochs = [0,1,2,3,4,5,6,7,8,9],
    # Total number of allowable colours in palette.
    pallete_size = 8,
    # Number of colours allocated to the heatmap.
    heatmap_palette_size = 6,
    # Offset to reach those colours (0 is reserved for no color, and 1 is reserved for ball present)
    heatmap_palette_offset = 2,
    
    matrix_width = 64,
    matrix_length = 32,
    
    
    # === Simulation ===
    
    # Random state for simulation.
    seed=42,
    
    # The time in seconds between each frame in the simulation. Lower values reduce error.
    simulation_interval = 1 / 80,
    
    starting_pos = (0, 0), # None
    lateral_acceleration = 20,
    vertical_acceleration = 0,
    number_of_balls = 1,
    
    # Ball speed is random integer in this range.
    ball_lat_potential = 1300,
    ball_vert_potential = 500,
    randomize_potential = False,
    
    # === Training Parameters ===
    
    # The model be trained on only every <model_interval>th frame. Allows for lower simulation error.
    model_interval = 4,
    
    model = model,
    # How many frames can the model see?
    number_of_prev_frames = number_of_prev_frames,
    # Number of examples to generate.
    number_of_examples = 10000,
    epochs = 3,
    batch_size = 50,
    
    # Generate fresh training data for on-the-fly training.
    gen_during_training = True,
    # The number of threads to use when generating training data
    num_simulation_threads = 1,
    # Cache models during training.
    model_caching = False,
    
    optimizer = opt,
    scheduler = scheduler, # scheduler
    loss_fxn = loss,
    
    # === Evaluation Parameters ===
    
    # Show a probability heatmap rather than showing the most likely position of the ball.
    use_heatmap = False,
    # Re-create the simulation rather than use the training data. Allows for inifinite execution.
    new_simulation = True,
    # Whether to show the actual position of the ball in addition to the predicted position.
    show_actual = True
)
config.get_data_path().mkdir(parents=True, exist_ok=True)

# Generate Data

In [42]:
gen_data(config)

# Train Model

In [43]:
train(config)

Starting Epoch: 0


acc: 19/50 loss: 1.84: 100%|██████████| 200/200 [00:18<00:00, 10.75it/s]


Epoch Train Accuracy:  36.25%
Mean Loss : 0.03
Starting Epoch: 1


acc: 13/50 loss: 1.49: 100%|██████████| 200/200 [00:15<00:00, 13.18it/s]


Epoch Train Accuracy:  59.59%
Mean Loss : 0.03
Starting Epoch: 2


acc: 12/50 loss: 1.37: 100%|██████████| 200/200 [00:15<00:00, 13.29it/s]


Epoch Train Accuracy:  34.55%
Mean Loss : 0.03


# Visualize Model

In [7]:
evaluate(config)

Process Process-2:
Traceback (most recent call last):
  File "/Users/ben/miniforge3/envs/py312/lib/python3.12/site-packages/serial/serialposix.py", line 575, in read
    buf = os.read(self.fd, size - len(read))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 6] Device not configured

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ben/miniforge3/envs/py312/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/ben/Projects/bouncyBall/src/simulation.py", line 35, in run
    raise e  # You can still rise this exception if you need to
    ^^^^^^^
  File "/Users/ben/Projects/bouncyBall/src/simulation.py", line 30, in run
    mp.Process.run(self)
  File "/Users/ben/miniforge3/envs/py312/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ben/Projects/bouncyBall/src/simulation.py", line 55, in _rec

Traceback (most recent call last):
  File "/Users/ben/miniforge3/envs/py312/lib/python3.12/site-packages/serial/serialposix.py", line 575, in read
    buf = os.read(self.fd, size - len(read))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 6] Device not configured

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ben/Projects/bouncyBall/src/simulation.py", line 30, in run
    mp.Process.run(self)
  File "/Users/ben/miniforge3/envs/py312/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/ben/Projects/bouncyBall/src/simulation.py", line 55, in _receive_layers
    _update_from_layers(layers, s, num_colours)
  File "/Users/ben/Projects/bouncyBall/src/simulation.py", line 78, in _update_from_layers
    s.readline().decode()
    ^^^^^^^^^^^^
  File "/Users/ben/miniforge3/envs/py312/lib/python3.12/site-packages/serial/serialposix.py", line 5

KeyboardInterrupt: 